In [1]:
import pandas as pd 
import numpy as np
import datetime as dt

from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
import warnings
import requests
import re

import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

In [73]:
driver = webdriver.Chrome()

In [72]:
# NYTimes

from dateutil.relativedelta import relativedelta
import datetime as dt

stdt = dt.datetime(2018,9,10)
(stdt + relativedelta(months=1)).strftime('%Y%m%d')
while stdt < dt.datetime(2022,2,10):
    date_lst = []
    title_lst = []
    summary_lst = []
    
    start_date = stdt.strftime('%Y%m%d')
    stdt = stdt + relativedelta(months=1)
    end_date = stdt.strftime('%Y%m%d')
    section_lst = ['Business%7Cnyt%3A%2F%2Fsection%2F0415b2b0-513a-5e78-80da-21ab770cb753','World%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b']
    #https://www.nytimes.com/search?dropmab=true&endDate=20220214&query=oil%20price&sections=Business%7Cnyt%3A%2F%2Fsection%2F0415b2b0-513a-5e78-80da-21ab770cb753&sort=oldest&startDate=20220101&types=article
    for section in section_lst:
        url = f'https://www.nytimes.com/search?dropmab=true&endDate={end_date}&query=oil%20price&sections={section}&sort=oldest&startDate={start_date}&types=article'
        driver.get(url)
        time.sleep(0.5)

        while driver.find_elements_by_css_selector('div.css-vsuiox > button'):
            driver.find_elements_by_css_selector('div.css-vsuiox > button')[0].click()
            time.sleep(1.5)
        html = driver.page_source
        #print(html[:100])
        soup = BeautifulSoup(html, 'html.parser')
        for article in soup.select('#site-content > div > div:nth-child(2) > div > ol > li.css-1l4w6pd'):
            if article.select('div > div > p')[0].text == 'DealBook':
                continue
            title = article.select('div > div > a > h4')[0].text
            try :
                summary = article.select('div > div > a > p')[0].text
            except:
                summary = np.nan
            date = article.select('div > span')[0].text
            
            date_lst.append(date)
            title_lst.append(title)
            summary_lst.append(summary)
    df = pd.DataFrame({'date':date_lst,'title':title_lst, 'summary':summary_lst})
    df.to_csv(f'./crawling/NYTimes/news_NYT_{start_date}_{end_date}.csv')


In [ ]:
# WP
url = 'https://www.washingtonpost.com/search?query=oil%20price&btn-search=&facets=%7B%22time%22%3A%5B1640444400000%2C1644847002340%5D%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%22Business%22%5D%2C%22author%22%3A%5B%5D%7D'
driver.get(url)

In [139]:
from dateutil.relativedelta import relativedelta
import datetime as dt

stdt = dt.datetime(2021,2,10)
while stdt < dt.datetime(2022,2,10):
    start_date = stdt.strftime('%m/%d/%y')
    sd = stdt.strftime('%Y%m%d')
    stdt = stdt + relativedelta(months=1)
    end_date = stdt.strftime('%m/%d/%y')
    ed = stdt.strftime('%Y%m%d')
    
    title_lst = []
    summary_lst = []
    date_lst = []

    sections = ['World', 'Business','Politics']
    for section in sections:
        url = f'https://www.washingtonpost.com/search?query=oil%20price&btn-search=&facets=%7B%22time%22%3A%5B1640444400000%2C1644847002340%5D%2C%22sort%22%3A%22relevancy%22%2C%22section%22%3A%5B%22{section}%22%5D%2C%22author%22%3A%5B%5D%7D'
        driver.get(url)
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[2]/div[1]/button').click()
        time.sleep(0.2)
        driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[2]/div[1]/div/div/div[1]/button/div/div[2]').click()
        time.sleep(0.2)

        driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[2]/div[1]/div/div/div[2]/div[1]/div/div/div[1]/input').clear()
        driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[2]/div[1]/div/div/div[2]/div[1]/div/div/div[1]/input').send_keys(start_date)

        driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input').clear()
        driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[2]/div[1]/div/div/div[2]/div[2]/div/div/div[1]/input').send_keys(end_date)

        driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[2]/div[1]/div/div/div[2]/button[2]').click()
        time.sleep(1)

        while driver.find_elements_by_xpath('//*[@id="main-content"]/div[1]/section[3]/button'):
            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="main-content"]/div[1]/section[3]/button').click()
            time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        for article in soup.select('#main-content > div:nth-child(1) > section.search-results-wrapper > article'):
            title_lst.append(article.select('div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div > a')[0].text)
            summary_lst.append(article.select('div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div')[0].text)
            date_lst.append(article.select('div:nth-child(2) > div:nth-child(2) > div:nth-child(2) > span')[0].text)
    df = pd.DataFrame({'date':date_lst,'title':title_lst, 'summary':summary_lst})
    df.to_csv(f'./crawling/WashingtonPost/news_WP_{sd}_{ed}.csv')

In [185]:
# FT
stdt = dt.datetime(2017,2,10)
while stdt < dt.datetime(2022,2,10):
    start_date = stdt.strftime('%Y-%m-%d')
    sd = stdt.strftime('%Y%m%d')
    stdt = stdt + relativedelta(months=1)
    end_date = stdt.strftime('%Y-%m-%d')
    ed = stdt.strftime('%Y%m%d')

    url = f'https://www.ft.com/search?expandRefinements=true&q=oil+price&concept=a579350c-61ce-4c00-97ca-ddaa2e0cacf6&dateFrom={start_date}&dateTo={end_date}'
    soup = BeautifulSoup(requests.get(url).text,'html.parser')
    try:
        pages = int(soup.select('#site-content > div > div.search-results__pagination > div > span')[0].text.split()[-1])
    except:
        pages = 1
    title_lst = []
    summary_lst = []
    date_lst = []

    for pp in range(1, pages+1):

        url = f'https://www.ft.com/search?expandRefinements=true&q=oil+price&page={pp}&concept=a579350c-61ce-4c00-97ca-ddaa2e0cacf6&dateFrom={start_date}&dateTo={end_date}'
        res = requests.get(url)
        time.sleep(2)
        soup = BeautifulSoup(res.text,'html.parser')
        for article in soup.select('#site-content > div > ul > li'):
            try:
                date_lst.append(article.select('div > div > div > div:nth-child(1) > div.o-teaser__timestamp > time')[0].text)
            except:
                continue
            title_lst.append(article.select('div > div > div > div:nth-child(1) > div.o-teaser__heading')[0].text)
            try:
                summary_lst.append(article.select('div > div > div > div:nth-child(1) > p > a > span')[0].text)
            except:
                summary_lst.append([])
            
    
    df = pd.DataFrame({'date':date_lst,'title':title_lst, 'summary':summary_lst})
    df.to_csv(f'./crawling/FinancialTimes/news_FT_{sd}_{ed}.csv')



KeyboardInterrupt: 

In [ ]:
# The Wallstreet Journals

from dateutil.relativedelta import relativedelta
import datetime as dt
import time
import pandas as pd

stdt = dt.datetime(2020,6,10) # 스타트데이트타임 
(stdt + relativedelta(months=1)).strftime('%Y%m%d') # 날짜에서 한달
while stdt < dt.datetime(2021,11,10):
    date_lst = []
    title_lst = []
    summary_lst = []
    
    start_date = stdt.strftime('%Y/%m/%d')
    sd = stdt.strftime('%Y%m%d')
    stdt = stdt + relativedelta(months=1)
    end_date = stdt.strftime('%Y/%m/%d')
    ed = stdt.strftime('%Y%m%d')
    
    url = f'https://www.wsj.com/search?query=oil%20price&isToggleOn=true&operator=AND&sort=date-desc&duration=4y&startDate={start_date}&endDate={end_date}&source=wsjie%2Cblog%2Cwsjpro%2Cautowire'
    driver.get(url)
    time.sleep(5)
    try:
        pages = int(driver.find_elements_by_xpath('//*[@id="main"]/div[5]/div/div/div/span')[0].text.split()[1])
    except:
        pages = 1
    for page in range(1, pages+1):
        url = f'https://www.wsj.com/search?query=oil%20price&isToggleOn=true&operator=AND&sort=date-desc&duration=4y&startDate={start_date}&endDate={end_date}&source=wsjie%2Cblog%2Cwsjpro%2Cautowire&page={page}'
        driver.get(url)
        time.sleep(8)
        
        html = driver.page_source
        #print(html[:100])
        soup = BeautifulSoup(html, 'html.parser')
        for article in soup.select('#main > article'):
            title = article.select('div > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > h3 > a > span')[0].text
            try :
                summary = article.select('div > div:nth-child(1) > div:nth-child(1) > p > span')[0].text
            except:
                summary = np.nan
            try :
                date = article.select('div > div:nth-child(1) > div:nth-child(2) > div > p')[0].text
            except:
                date = np.nan
            
            date_lst.append(date)
            title_lst.append(title)
            summary_lst.append(summary)
    df = pd.DataFrame({'date':date_lst,'title':title_lst, 'summary':summary_lst})
    df.to_csv(f'crawling/TheWallStreetJournal/news_wsj_{sd}_{ed}.csv')

    